# Simulation

In [28]:
import simpy
import random as rnd
import statistics as st

### Define simulation: environment, resources and timing

In [37]:
class Cinema():
    def __init__(self, env, ncashiers, nservers, nushers):
        self.env = simpy.Environment()
        self.cashier = simpy.Resource(env, ncashiers)
        self.server = simpy.Resource(env, nservers)
        self.usher = simpy.Resource(env, nushers)

    def purchase_ticket(self, customer):
        yield self.env.timeout(rnd.randint(1, 3))

    def check_ticket(self, customer):
        yield self.env.timeout(3/60)

    def sell_food(self, customer):
        yield self.env.timeout(rnd.randint(1, 5))

### Define action and general process

In [35]:
def go_cinema(customer, cinema):
    env = cinema.env
    arrival_time = env.now

    with cinema.cashier.request() as request:
        yield request
        yield env.process(cinema.purchase_ticket(customer))

    with cinema.usher.request() as request:
        yield request
        yield env.process(cinema.check_ticket(customer))

    if random.choice([True, False]):
        with cinema.server.request() as request:
            yield request
            yield env.process(cinema.sell_food(customer))

    # Customer heads into the theater
    wait_times.append(env.now - arrival_time)


def cinema_process(cinema, ncustomers):
    
    for customer in range(ncustomers):
        env.process(go_cinema(customer, cinema))

    while True:
        yield env.timeout(0.20)  # Wait a bit before generating a new person

        customer += 1
        env.process(go_cinema(customer, cinema))


def get_waittime(wait_times):
    avg_wait = st.mean(wait_times)
    mins, frac_min = divmod(avg_wait, 1)
    secs = frac_min * 60
    return round(mins), round(secs)

### Run simulation and get results

In [36]:
cinema = Cinema(env, ncashiers=3, nservers=2, nushers=5)

cinema.env.process(cinema_process(cinema=cinema, ncustomers=3))
cinema.env.run(until=90)

mins, secs = get_avg_waittime(wait_times)
print('Running simulation... The average wait time is ', mins, ' minutes and ', secs ,' seconds.')

Running simulation... The average wait time is  33  minutes and  44  seconds.


### Credits & Links

https://simpy.readthedocs.io/en/latest/examples/movie_renege.html  
https://realpython.com/simpy-simulating-with-python/  
http://users.iems.northwestern.edu/~nelsonb/IEMS435/PythonSim.pdf